# 前置設定

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
# import argparse
from torch.utils.data import DataLoader

from drive.MyDrive.iirlab.topic.LSAN.LSANori.trainer import *
from drive.MyDrive.iirlab.topic.LSAN.LSANori.transformer import *
from drive.MyDrive.iirlab.topic.LSAN.LSANori.dataset import *
from drive.MyDrive.iirlab.topic.LSAN.LSANori.LSAN import *

import pickle

In [3]:
time_interval = '12_with_gap_6/18_39'
with open('/content/drive/MyDrive/iirlab/topic/trimmed_pickle/' + time_interval + '/final_train.pickle','rb') as f:
  train_data = pickle.load(f)
  train_visit = train_data[0]
  train_label = train_data[1]
  train_value = train_data[3]
  train_dataset = Dataset(train_visit, train_label, train_value)
  

with open('/content/drive/MyDrive/iirlab/topic/trimmed_pickle/' + time_interval + '/final_val.pickle','rb') as f:
  validate_data = pickle.load(f)
  validate_visit = validate_data[0]
  validate_label = validate_data[1]
  validate_value = validate_data[3]
  validate_dataset = Dataset(validate_visit, validate_label, validate_value)
  

with open('/content/drive/MyDrive/iirlab/topic/trimmed_pickle/' + time_interval + '/final_test.pickle','rb') as f:
  test_data = pickle.load(f)
  test_visit = test_data[0]
  test_label = test_data[1]
  test_value = test_data[3]
  test_dataset = Dataset(test_visit, test_label, test_value)
  

# Setting new dataset and dataloader
# with open('/content/drive/MyDrive/iirlab/topic/LSAN/LSANori/model_inputs/' + time_interval + '/final_train.pickle','rb') as f:
#   train_data = pickle.load(f)
#   train_visit = train_data[0]
#   train_label = train_data[1]
#   train_value = train_data[3]
#   train_dataset = Dataset(train_visit, train_label, train_value)

# with open('/content/drive/MyDrive/iirlab/topic/LSAN/LSANori/model_inputs/' + time_interval + '/final_val.pickle','rb') as f:
#   validate_data = pickle.load(f)
#   validate_visit = validate_data[0]
#   validate_label = validate_data[1]
#   validate_value = validate_data[3]
#   validate_dataset = Dataset(validate_visit, validate_label, validate_value)

# with open('/content/drive/MyDrive/iirlab/topic/LSAN/LSANori/model_inputs/' + time_interval + '/final_test.pickle','rb') as f:
#   test_data = pickle.load(f)
#   test_visit = test_data[0]
#   test_label = test_data[1]
#   test_value = test_data[3]
#   test_dataset = Dataset(test_visit, test_label, test_value)


# 參數

In [4]:
output_path = '/content/drive/MyDrive/iirlab/topic/LSAN/LSANori/output/result.txt'
hidden = 256 
layers = 8
attn_heads = 8 
dropout = 0.1
batch_size = 32
epochs = 200
num_workers = 2
with_cuda = 1
lr = 0.0001
adam_weight_decay = 0.01 ###
adam_beta1 = 0.9
adam_beta2 = 0.999
saving_path = '/content/drive/MyDrive/iirlab/topic/LSAN/LSANori/model_parameters/models.pth'

In [5]:
def process():
  # print("Creating Dataloader")
  train_data_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers, collate_fn=train_dataset.collate_fn,shuffle=False)
  validate_data_loader = DataLoader(validate_dataset, batch_size=batch_size, num_workers=num_workers, collate_fn=test_dataset.collate_fn,shuffle=False)
  test_data_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=num_workers, collate_fn=test_dataset.collate_fn,shuffle=False)




  embedding_dim = hidden
  code2idx_file = '/content/drive/MyDrive/iirlab/topic/LSAN/LSANori/code2idx_new.pickle'



  with open(code2idx_file, 'rb') as f:
      code2idx = pickle.load(f)
      #  list out keys and values separately
      diagnosis_code_list = list(code2idx.keys())
      dignosis_index_list = list(code2idx.values())
      # print(len(dignosis_index_list))

  # model = LSAN(len(dignosis_index_list), embedding_dim, transformer_hidden = hidden, attn_heads = attn_heads,
  #                     transformer_dropout = dropout, transformer_layers = layers)
  model = LSAN(51, embedding_dim, transformer_hidden = hidden, attn_heads = attn_heads,
                      transformer_dropout = dropout, transformer_layers = layers)

  # defining trainer
  LSAN_train = LSAN_trainer(model, train_dataloader=train_data_loader, validate_dataloader=validate_data_loader, test_dataloader=test_data_loader, with_cuda=with_cuda, lr=lr, output_dir=output_path)

  # training process
  for epoch in range(epochs):
      model.train()
      LSAN_train.train(epoch)
  
      # Validation
      if validate_data_loader is not None:
          model.eval()
          LSAN_train.validate(epoch)
        
  torch.save(model.state_dict(), saving_path)

  # Test after the end of training
  # test_model = LSAN(len(dignosis_index_list), embedding_dim, transformer_hidden = hidden, attn_heads = attn_heads, transformer_dropout = dropout, transformer_layers = layers) 
  test_model = LSAN(51, embedding_dim, transformer_hidden = hidden, attn_heads = attn_heads, transformer_dropout = dropout, transformer_layers = layers) 


  test_model.load_state_dict(torch.load(saving_path))
  test_model.eval()

  LSAN_test = LSAN_trainer(test_model, train_dataloader=train_data_loader, validate_dataloader=validate_data_loader, test_dataloader=test_data_loader, with_cuda=with_cuda, lr=lr, output_dir=output_path)

  LSAN_test.test(epoch)

# 實驗1

In [6]:
output_path = '/content/drive/MyDrive/iirlab/topic/LSAN/LSANori/output/result.txt'
hidden = 256 
layers = 8
attn_heads = 8 
dropout = 0.1
batch_size = 32
epochs = 200
num_workers = 2
with_cuda = 1
lr = 0.0001
adam_weight_decay = 0.01 ###
adam_beta1 = 0.9
adam_beta2 = 0.999
saving_path = '/content/drive/MyDrive/iirlab/topic/LSAN/LSANori/model_parameters/models.pth'

def process():
  # print("Creating Dataloader")
  train_data_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers, collate_fn=train_dataset.collate_fn,shuffle=False)
  validate_data_loader = DataLoader(validate_dataset, batch_size=batch_size, num_workers=num_workers, collate_fn=test_dataset.collate_fn,shuffle=False)
  test_data_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=num_workers, collate_fn=test_dataset.collate_fn,shuffle=False)




  embedding_dim = hidden
  code2idx_file = '/content/drive/MyDrive/iirlab/topic/LSAN/LSANori/code2idx_new.pickle'



  with open(code2idx_file, 'rb') as f:
      code2idx = pickle.load(f)
      #  list out keys and values separately
      diagnosis_code_list = list(code2idx.keys())
      dignosis_index_list = list(code2idx.values())
      # print(len(dignosis_index_list))

  # model = LSAN(len(dignosis_index_list), embedding_dim, transformer_hidden = hidden, attn_heads = attn_heads,
  #                     transformer_dropout = dropout, transformer_layers = layers)
  model = LSAN(51, embedding_dim, transformer_hidden = hidden, attn_heads = attn_heads,
                      transformer_dropout = dropout, transformer_layers = layers)

  # defining trainer
  LSAN_train = LSAN_trainer(model, train_dataloader=train_data_loader, validate_dataloader=validate_data_loader, test_dataloader=test_data_loader, with_cuda=with_cuda, lr=lr, output_dir=output_path)

  # training process
  for epoch in range(epochs):
      model.train()
      LSAN_train.train(epoch)
  
      # Validation
      if validate_data_loader is not None:
          model.eval()
          LSAN_train.validate(epoch)
        
  torch.save(model.state_dict(), saving_path)

  # Test after the end of training
  # test_model = LSAN(len(dignosis_index_list), embedding_dim, transformer_hidden = hidden, attn_heads = attn_heads, transformer_dropout = dropout, transformer_layers = layers) 
  test_model = LSAN(51, embedding_dim, transformer_hidden = hidden, attn_heads = attn_heads, transformer_dropout = dropout, transformer_layers = layers) 


  test_model.load_state_dict(torch.load(saving_path))
  test_model.eval()

  LSAN_test = LSAN_trainer(test_model, train_dataloader=train_data_loader, validate_dataloader=validate_data_loader, test_dataloader=test_data_loader, with_cuda=with_cuda, lr=lr, output_dir=output_path)

  LSAN_test.test(epoch)

In [ ]:
epochs = 10
process()

Done for epoch 0
0.7594562931721952
Validation epoch:0 auc:0.5382914067104839 precision:0.6894024802705749 recall:0.9894822006472492
Validation loss : 0.6808750179775974
Done for epoch 1
0.6455173886874143
Validation epoch:1 auc:0.7894632947430466 precision:0.8 recall:0.883495145631068
Validation loss : 0.5106369544539535
Done for epoch 2
0.5728428185987873
Validation epoch:2 auc:0.807371078415018 precision:0.8043956043956044 recall:0.8883495145631068
Validation loss : 0.49271604941602337
Done for epoch 3
0.5540914477921334
Validation epoch:3 auc:0.8158342508257432 precision:0.8113485630066323 recall:0.8907766990291263
Validation loss : 0.4814686341243878
Done for epoch 4
0.5431670170872152
Validation epoch:4 auc:0.8172674851811075 precision:0.8108308605341247 recall:0.8843042071197411
Validation loss : 0.4790811313871752
Done for epoch 5
0.5366417159553335
Validation epoch:5 auc:0.8192387037222388 precision:0.8191653786707882 recall:0.8576051779935275
Validation loss : 0.4796627404396

In [ ]:
epochs = 35
process()

Done for epoch 0
0.65483972580493
Validation epoch:0 auc:0.7877562028047466 precision:0.7975415762834418 recall:0.8923948220064725
Validation loss : 0.5107700411687818
Done for epoch 1
0.5768995434045792
Validation epoch:1 auc:0.8009305597259757 precision:0.804138950480414 recall:0.8802588996763754
Validation loss : 0.49885740562489156
Done for epoch 2
0.5636866609839832
Validation epoch:2 auc:0.8071125123722461 precision:0.8072646404744255 recall:0.8810679611650486
Validation loss : 0.49724586438714413
Done for epoch 3
0.5568906197778317
Validation epoch:3 auc:0.8111953535960142 precision:0.8093093093093093 recall:0.872168284789644
Validation loss : 0.4934466779232025
Done for epoch 4
0.5495209165480959
Validation epoch:4 auc:0.8138254985042094 precision:0.8111361926260346 recall:0.872168284789644
Validation loss : 0.48914886932624013
Done for epoch 5
0.5422964583174521
Validation epoch:5 auc:0.8160302605678443 precision:0.8097378277153559 recall:0.8745954692556634
Validation loss : 0

# 實驗2 
嘗試將學習率降低，期望能使開始overfitting的時間點變得更後面(之後可銜接weight_decay的實驗)

In [ ]:
output_path = '/content/drive/MyDrive/iirlab/topic/LSAN/LSANori/output/result.txt'
hidden = 256 
layers = 8
attn_heads = 8 
dropout = 0.1
batch_size = 32
epochs = 200
num_workers = 2
with_cuda = 1
lr = 0.00005
adam_weight_decay = 0.01 ###
adam_beta1 = 0.9
adam_beta2 = 0.999
saving_path = '/content/drive/MyDrive/iirlab/topic/LSAN/LSANori/model_parameters/models.pth'

def process():
  # print("Creating Dataloader")
  train_data_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers, collate_fn=train_dataset.collate_fn,shuffle=False)
  validate_data_loader = DataLoader(validate_dataset, batch_size=batch_size, num_workers=num_workers, collate_fn=test_dataset.collate_fn,shuffle=False)
  test_data_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=num_workers, collate_fn=test_dataset.collate_fn,shuffle=False)




  embedding_dim = hidden
  code2idx_file = '/content/drive/MyDrive/iirlab/topic/LSAN/LSANori/code2idx_new.pickle'



  with open(code2idx_file, 'rb') as f:
      code2idx = pickle.load(f)
      #  list out keys and values separately
      diagnosis_code_list = list(code2idx.keys())
      dignosis_index_list = list(code2idx.values())
      # print(len(dignosis_index_list))

  # model = LSAN(len(dignosis_index_list), embedding_dim, transformer_hidden = hidden, attn_heads = attn_heads,
  #                     transformer_dropout = dropout, transformer_layers = layers)
  model = LSAN(51, embedding_dim, transformer_hidden = hidden, attn_heads = attn_heads,
                      transformer_dropout = dropout, transformer_layers = layers)

  # defining trainer
  LSAN_train = LSAN_trainer(model, train_dataloader=train_data_loader, validate_dataloader=validate_data_loader, test_dataloader=test_data_loader, with_cuda=with_cuda, lr=lr, output_dir=output_path)

  # training process
  for epoch in range(epochs):
      model.train()
      LSAN_train.train(epoch)
  
      # Validation
      if validate_data_loader is not None:
          model.eval()
          LSAN_train.validate(epoch)
        
  torch.save(model.state_dict(), saving_path)

  # Test after the end of training
  # test_model = LSAN(len(dignosis_index_list), embedding_dim, transformer_hidden = hidden, attn_heads = attn_heads, transformer_dropout = dropout, transformer_layers = layers) 
  test_model = LSAN(51, embedding_dim, transformer_hidden = hidden, attn_heads = attn_heads, transformer_dropout = dropout, transformer_layers = layers) 


  test_model.load_state_dict(torch.load(saving_path))
  test_model.eval()

  LSAN_test = LSAN_trainer(test_model, train_dataloader=train_data_loader, validate_dataloader=validate_data_loader, test_dataloader=test_data_loader, with_cuda=with_cuda, lr=lr, output_dir=output_path)

  LSAN_test.test(epoch)

In [ ]:
epochs = 20
process()

In [ ]:
epochs = 20
process()

In [ ]:
epochs = 50
process()

In [ ]:
epochs = 50
process()

In [ ]:
epochs = 100
process()

In [ ]:
epochs = 100
process()

# 實驗3
batch size

In [ ]:
output_path = '/content/drive/MyDrive/iirlab/topic/LSAN/LSANori/output/result.txt'
hidden = 256 
layers = 8
attn_heads = 8 
dropout = 0.1
batch_size = 64
epochs = 200
num_workers = 2
with_cuda = 1
lr = 0.00005
adam_weight_decay = 0.01 
adam_beta1 = 0.9
adam_beta2 = 0.999
saving_path = '/content/drive/MyDrive/iirlab/topic/LSAN/LSANori/model_parameters/models.pth'

def process():
  # print("Creating Dataloader")
  train_data_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers, collate_fn=train_dataset.collate_fn,shuffle=False)
  validate_data_loader = DataLoader(validate_dataset, batch_size=batch_size, num_workers=num_workers, collate_fn=test_dataset.collate_fn,shuffle=False)
  test_data_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=num_workers, collate_fn=test_dataset.collate_fn,shuffle=False)




  embedding_dim = hidden
  code2idx_file = '/content/drive/MyDrive/iirlab/topic/LSAN/LSANori/code2idx_new.pickle'



  with open(code2idx_file, 'rb') as f:
      code2idx = pickle.load(f)
      #  list out keys and values separately
      diagnosis_code_list = list(code2idx.keys())
      dignosis_index_list = list(code2idx.values())
      # print(len(dignosis_index_list))

  # model = LSAN(len(dignosis_index_list), embedding_dim, transformer_hidden = hidden, attn_heads = attn_heads,
  #                     transformer_dropout = dropout, transformer_layers = layers)
  model = LSAN(51, embedding_dim, transformer_hidden = hidden, attn_heads = attn_heads,
                      transformer_dropout = dropout, transformer_layers = layers)

  # defining trainer
  LSAN_train = LSAN_trainer(model, train_dataloader=train_data_loader, validate_dataloader=validate_data_loader, test_dataloader=test_data_loader, with_cuda=with_cuda, lr=lr, output_dir=output_path)

  # training process
  for epoch in range(epochs):
      model.train()
      LSAN_train.train(epoch)
  
      # Validation
      if validate_data_loader is not None:
          model.eval()
          LSAN_train.validate(epoch)
        
  torch.save(model.state_dict(), saving_path)

  # Test after the end of training
  # test_model = LSAN(len(dignosis_index_list), embedding_dim, transformer_hidden = hidden, attn_heads = attn_heads, transformer_dropout = dropout, transformer_layers = layers) 
  test_model = LSAN(51, embedding_dim, transformer_hidden = hidden, attn_heads = attn_heads, transformer_dropout = dropout, transformer_layers = layers) 


  test_model.load_state_dict(torch.load(saving_path))
  test_model.eval()

  LSAN_test = LSAN_trainer(test_model, train_dataloader=train_data_loader, validate_dataloader=validate_data_loader, test_dataloader=test_data_loader, with_cuda=with_cuda, lr=lr, output_dir=output_path)

  LSAN_test.test(epoch)

In [ ]:
batch_size = 64
epochs = 20
process()

In [ ]:
batch_size = 64
epochs = 50
process()

In [ ]:
batch_size = 64
epochs = 100
process()

In [ ]:
batch_size = 128
epochs = 20
process()

Done for epoch 0
0.8631143540143966
Validation epoch:0 auc:0.45760907038556925 precision:0.6798679867986799 recall:1.0
Validation loss : 0.6518922090530396
Done for epoch 1
0.6977490355571111
Validation epoch:1 auc:0.5435996563573884 precision:0.6831971995332555 recall:0.9474110032362459
Validation loss : 0.6653458515803019
Done for epoch 2
0.6945037126541138
Validation epoch:2 auc:0.6947266428674695 precision:0.6844148641153632 recall:0.9983818770226537
Validation loss : 0.6564481457074484
Done for epoch 3
0.6681034247080485
Validation epoch:3 auc:0.7410294264838355 precision:0.7837837837837838 recall:0.8915857605177994
Validation loss : 0.5930088559786478
Done for epoch 4
0.6137826422850291
Validation epoch:4 auc:0.7786674673873153 precision:0.800599700149925 recall:0.8640776699029126
Validation loss : 0.5532180627187093
Done for epoch 5
0.5870786339044571
Validation epoch:5 auc:0.7866635527530332 precision:0.8045801526717558 recall:0.8527508090614887
Validation loss : 0.532304114103

In [ ]:
batch_size = 128
epochs = 50
process()

Done for epoch 0
0.8742616613705952
Validation epoch:0 auc:0.6844548982973564 precision:0.9019607843137255 recall:0.0372168284789644
Validation loss : 0.7054200053215027
Done for epoch 1
0.6841529111067454
Validation epoch:1 auc:0.726763809650908 precision:0.8327402135231317 recall:0.3786407766990291
Validation loss : 0.6852998256683349
Done for epoch 2
0.6522781044244766
Validation epoch:2 auc:0.7443310090192284 precision:0.7934782608695652 recall:0.8268608414239482
Validation loss : 0.581907327969869
Done for epoch 3
0.6097018361091614
Validation epoch:3 auc:0.7705254173200327 precision:0.8018575851393189 recall:0.8381877022653722
Validation loss : 0.5561637918154398
Done for epoch 4
0.5936360438664754
Validation epoch:4 auc:0.7807151436292663 precision:0.8047839506172839 recall:0.8438511326860841
Validation loss : 0.5355329612890879
Done for epoch 5
0.5848333825667699
Validation epoch:5 auc:0.784485898419689 precision:0.8088578088578089 recall:0.8422330097087378
Validation loss : 0.

In [ ]:
batch_size = 128
epochs = 100
process()

Done for epoch 0
0.7167219112316767
Validation epoch:0 auc:0.6775528531233665 precision:0.9119170984455959 recall:0.1423948220064725
Validation loss : 0.6971226771672566
Done for epoch 1
0.6820355296134949
Validation epoch:1 auc:0.7475783760940402 precision:0.7886557886557887 recall:0.8211974110032363
Validation loss : 0.6486051162083943
Done for epoch 2
0.6258647948503494
Validation epoch:2 auc:0.7699359979537139 precision:0.8025914634146342 recall:0.8519417475728155
Validation loss : 0.5411114970842997
Done for epoch 3
0.5865398039420445
Validation epoch:3 auc:0.7862923853690544 precision:0.8063271604938271 recall:0.8454692556634305
Validation loss : 0.5236473838488261
Done for epoch 4
0.5755204250415166
Validation epoch:4 auc:0.7937449259889455 precision:0.8092664092664092 recall:0.8478964401294499
Validation loss : 0.5164784570535024
Done for epoch 5
0.5700471897919973
Validation epoch:5 auc:0.7993152170286592 precision:0.8128383604021655 recall:0.8503236245954693
Validation loss :

# 實驗4 
調整hidden個數

In [ ]:
output_path = '/content/drive/MyDrive/iirlab/topic/LSAN/LSANori/output/result.txt'
hidden = 256 
layers = 8
attn_heads = 8 
dropout = 0.1
batch_size = 32
epochs = 200
num_workers = 2
with_cuda = 1
lr = 0.00005
adam_weight_decay = 0.01 ###
adam_beta1 = 0.9
adam_beta2 = 0.999
saving_path = '/content/drive/MyDrive/iirlab/topic/LSAN/LSANori/model_parameters/models.pth'

def process():
  # print("Creating Dataloader")
  train_data_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers, collate_fn=train_dataset.collate_fn,shuffle=False)
  validate_data_loader = DataLoader(validate_dataset, batch_size=batch_size, num_workers=num_workers, collate_fn=test_dataset.collate_fn,shuffle=False)
  test_data_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=num_workers, collate_fn=test_dataset.collate_fn,shuffle=False)




  embedding_dim = hidden
  code2idx_file = '/content/drive/MyDrive/iirlab/topic/LSAN/LSANori/code2idx_new.pickle'



  with open(code2idx_file, 'rb') as f:
      code2idx = pickle.load(f)
      #  list out keys and values separately
      diagnosis_code_list = list(code2idx.keys())
      dignosis_index_list = list(code2idx.values())
      # print(len(dignosis_index_list))

  # model = LSAN(len(dignosis_index_list), embedding_dim, transformer_hidden = hidden, attn_heads = attn_heads,
  #                     transformer_dropout = dropout, transformer_layers = layers)
  model = LSAN(51, embedding_dim, transformer_hidden = hidden, attn_heads = attn_heads,
                      transformer_dropout = dropout, transformer_layers = layers)

  # defining trainer
  LSAN_train = LSAN_trainer(model, train_dataloader=train_data_loader, validate_dataloader=validate_data_loader, test_dataloader=test_data_loader, with_cuda=with_cuda, lr=lr, output_dir=output_path)

  # training process
  for epoch in range(epochs):
      model.train()
      LSAN_train.train(epoch)
  
      # Validation
      if validate_data_loader is not None:
          model.eval()
          LSAN_train.validate(epoch)
        
  torch.save(model.state_dict(), saving_path)

  # Test after the end of training
  # test_model = LSAN(len(dignosis_index_list), embedding_dim, transformer_hidden = hidden, attn_heads = attn_heads, transformer_dropout = dropout, transformer_layers = layers) 
  test_model = LSAN(51, embedding_dim, transformer_hidden = hidden, attn_heads = attn_heads, transformer_dropout = dropout, transformer_layers = layers) 


  test_model.load_state_dict(torch.load(saving_path))
  test_model.eval()

  LSAN_test = LSAN_trainer(test_model, train_dataloader=train_data_loader, validate_dataloader=validate_data_loader, test_dataloader=test_data_loader, with_cuda=with_cuda, lr=lr, output_dir=output_path)

  LSAN_test.test(epoch)

In [ ]:
hidden = 512 
epochs = 20
process()

Done for epoch 0
0.7845091596871865
Validation epoch:0 auc:0.5948346567466275 precision:0.6798679867986799 recall:1.0
Validation loss : 0.6594797874751844
Done for epoch 1
0.6978231350413892
Validation epoch:1 auc:0.6095277416341374 precision:0.6798679867986799 recall:1.0
Validation loss : 0.665211650363186
Done for epoch 2
0.6971085592478263
Validation epoch:2 auc:0.6309372880036477 precision:0.6798679867986799 recall:1.0
Validation loss : 0.6722468043628492
Done for epoch 3
0.6975386473311096
Validation epoch:3 auc:0.5711223434424315 precision:0.6798679867986799 recall:1.0
Validation loss : 0.683567063850269
Done for epoch 4
0.6973864959568536
Validation epoch:4 auc:0.575332382477563 precision:0.6798679867986799 recall:1.0
Validation loss : 0.6808337245071143
Done for epoch 5
0.697460436770896
Validation epoch:5 auc:0.599124628832616 precision:0.6798679867986799 recall:1.0
Validation loss : 0.6848669062580979
Done for epoch 6
0.6962111577266404
Validation epoch:6 auc:0.59792076757971

In [ ]:
hidden = 512
epochs = 50
process()

Done for epoch 0
0.722128045909545
Validation epoch:0 auc:0.385265488940046 precision:0.6798679867986799 recall:1.0
Validation loss : 0.6674982832189191
Done for epoch 1
0.6982407552354476
Validation epoch:1 auc:0.37815770304385055 precision:0.6798679867986799 recall:1.0
Validation loss : 0.6681297003177175
Done for epoch 2
0.6978846282518211
Validation epoch:2 auc:0.3782105283644168 precision:0.6798679867986799 recall:1.0
Validation loss : 0.6713987473855939
Done for epoch 3
0.6975176777659344
Validation epoch:3 auc:0.397573788631991 precision:0.6798679867986799 recall:1.0
Validation loss : 0.6811116881537855
Done for epoch 4
0.6978075015945595
Validation epoch:4 auc:0.3631198634326449 precision:0.6798679867986799 recall:1.0
Validation loss : 0.6852015434649953
Done for epoch 5
0.6962779299551699
Validation epoch:5 auc:0.3556263136823141 precision:0.6798679867986799 recall:1.0
Validation loss : 0.6840899900386208
Done for epoch 6
0.6973530041570423
Validation epoch:6 auc:0.37177348502

In [ ]:
hidden = 512 
epochs = 100
process()

Done for epoch 0
0.6950888817801195
Validation epoch:0 auc:0.7840153360246443 precision:0.7985507246376812 recall:0.8915857605177994
Validation loss : 0.5102032893582394
Done for epoch 1
0.5784941490958718
Validation epoch:1 auc:0.7970492888043684 precision:0.8035581912527798 recall:0.8770226537216829
Validation loss : 0.5006143074286612
Done for epoch 2
0.5683212058634317
Validation epoch:2 auc:0.8027599839855869 precision:0.8048780487804879 recall:0.8810679611650486
Validation loss : 0.49355378119569077
Done for epoch 3
0.5603179140251224
Validation epoch:3 auc:0.8089141338315595 precision:0.8107908351810791 recall:0.8875404530744336
Validation loss : 0.4883462969671216
Done for epoch 4
0.5534836442029777
Validation epoch:4 auc:0.8136767535226147 precision:0.8148425787106447 recall:0.8794498381877023
Validation loss : 0.4863925921289544
Done for epoch 5
0.5440836278330378
Validation epoch:5 auc:0.8182294620714199 precision:0.8255722178374112 recall:0.8462783171521036
Validation loss 

In [ ]:
hidden = 1024
epochs = 20
process()

Done for epoch 0
1.453837312695359
Validation epoch:0 auc:0.7859990658259101 precision:0.8270609318996416 recall:0.7467637540453075
Validation loss : 0.5763456753471441
Done for epoch 1
0.583486976868966
Validation epoch:1 auc:0.7991692523270943 precision:0.8210609659540776 recall:0.8389967637540453
Validation loss : 0.5294483654331743
Done for epoch 2
0.5685494590957626
Validation epoch:2 auc:0.8107838721516031 precision:0.8198412698412698 recall:0.8357605177993528
Validation loss : 0.5188621810653753
Done for epoch 3
0.5533866936168751
Validation epoch:3 auc:0.8182989690721649 precision:0.8421052631578947 recall:0.7637540453074434
Validation loss : 0.5210843431322199
Done for epoch 4
0.5400501897855967
Validation epoch:4 auc:0.8221948364639287 precision:0.835696413678065 recall:0.8106796116504854
Validation loss : 0.4923838580909528
Done for epoch 5
0.5295668134168416
Validation epoch:5 auc:0.8229781803623262 precision:0.8383333333333334 recall:0.813915857605178
Validation loss : 0.4

In [ ]:
hidden = 1024
epochs = 50
process()

Done for epoch 0
0.6758419495670736
Validation epoch:0 auc:0.7913858583836564 precision:0.8111285266457681 recall:0.837378640776699
Validation loss : 0.5285935543085399
Done for epoch 1
0.5732299917134918
Validation epoch:1 auc:0.809931716322468 precision:0.8172549019607843 recall:0.843042071197411
Validation loss : 0.5180552910294449
Done for epoch 2
0.5578224508702254
Validation epoch:2 auc:0.8187076702365462 precision:0.8213457076566125 recall:0.8592233009708737
Validation loss : 0.503180348036582
Done for epoch 3
0.5452486772497162
Validation epoch:3 auc:0.8215213136267085 precision:0.8271213322759714 recall:0.8438511326860841
Validation loss : 0.4961468274133247
Done for epoch 4
0.5358870483997489
Validation epoch:4 auc:0.8245532089992104 precision:0.8278041074249605 recall:0.8478964401294499
Validation loss : 0.48784804082753364
Done for epoch 5
0.5281657759381944
Validation epoch:5 auc:0.8262046953369142 precision:0.8289578361177407 recall:0.843042071197411
Validation loss : 0.4

In [ ]:
hidden = 1024
epochs = 100
process()

Done for epoch 0
50.0
Validation epoch:0 auc:0.5 precision:0.6798679867986799 recall:1.0
Validation loss : 32.03441298635382
Done for epoch 1
50.0
Validation epoch:1 auc:0.5 precision:0.6798679867986799 recall:1.0
Validation loss : 32.03441298635382
Done for epoch 2
50.0
Validation epoch:2 auc:0.5 precision:0.6798679867986799 recall:1.0
Validation loss : 32.03441298635382
Done for epoch 3
50.0
Validation epoch:3 auc:0.5 precision:0.6798679867986799 recall:1.0
Validation loss : 32.03441298635382
Done for epoch 4
50.0
Validation epoch:4 auc:0.5 precision:0.6798679867986799 recall:1.0
Validation loss : 32.03441298635382
Done for epoch 5
50.0
Validation epoch:5 auc:0.5 precision:0.6798679867986799 recall:1.0
Validation loss : 32.03441298635382
Done for epoch 6
50.0
Validation epoch:6 auc:0.5 precision:0.6798679867986799 recall:1.0
Validation loss : 32.03441298635382
Done for epoch 7
50.0
Validation epoch:7 auc:0.5 precision:0.6798679867986799 recall:1.0
Validation loss : 32.03441298635382


In [ ]:
hidden = 2048
epochs = 20
process()

Done for epoch 0
0.7084896746052414
Validation epoch:0 auc:0.7948459168807482 precision:0.8062593144560357 recall:0.8754045307443366
Validation loss : 0.49762467856992754
Done for epoch 1
0.5791280328476128
Validation epoch:1 auc:0.8066634971474326 precision:0.8118279569892473 recall:0.8551779935275081
Validation loss : 0.4979149558042225
Done for epoch 2
0.5605444833010185
Validation epoch:2 auc:0.8181029593300637 precision:0.8336 recall:0.843042071197411
Validation loss : 0.4999041353401385
Done for epoch 3
0.5520518191722261
Validation epoch:3 auc:0.8219008218507768 precision:0.8337408312958435 recall:0.8276699029126213
Validation loss : 0.4924493212448923
Done for epoch 4
0.5423174788471029
Validation epoch:4 auc:0.8233980026468266 precision:0.8268467037331215 recall:0.8422330097087378
Validation loss : 0.48527236518106964
Done for epoch 5
0.5373764571522465
Validation epoch:5 auc:0.8239665699129216 precision:0.8326661329063251 recall:0.8414239482200647
Validation loss : 0.48540003

In [ ]:
hidden = 2048
epochs = 50
process()

Done for epoch 0
1.000373669281727
Validation epoch:0 auc:0.47679439273123586 precision:0.6798679867986799 recall:1.0
Validation loss : 0.6582654045339216
Done for epoch 1
0.7127017889704023
Validation epoch:1 auc:0.5015089969861766 precision:0.6798679867986799 recall:1.0
Validation loss : 0.6378417736605594
Done for epoch 2
0.7095227920207656
Validation epoch:2 auc:0.49502051846662004 precision:0.6798679867986799 recall:1.0
Validation loss : 0.6496684791748983
Done for epoch 3
0.7092221171415153
Validation epoch:3 auc:0.4698131095763965 precision:0.6798679867986799 recall:1.0
Validation loss : 0.6649776404364067
Done for epoch 4
0.7064892006020586
Validation epoch:4 auc:0.4659881393253929 precision:0.6798679867986799 recall:1.0
Validation loss : 0.6628678712928504
Done for epoch 5
0.7068869090881669
Validation epoch:5 auc:0.4849343575884963 precision:0.6798679867986799 recall:1.0
Validation loss : 0.6709848569150556
Done for epoch 6
0.7053143710148435
Validation epoch:6 auc:0.48279423

In [ ]:
hidden = 2048
epochs = 100
process()

# 實驗5
將hidden attenhead layer同時調高

In [ ]:
output_path = '/content/drive/MyDrive/iirlab/topic/LSAN/LSANori/output/result.txt'
hidden = 256 
layers = 8
attn_heads = 8 
dropout = 0.1
batch_size = 32
epochs = 200
num_workers = 2
with_cuda = 1
lr = 0.00005
adam_weight_decay = 0.01 ###
adam_beta1 = 0.9
adam_beta2 = 0.999
saving_path = '/content/drive/MyDrive/iirlab/topic/LSAN/LSANori/model_parameters/models.pth'

def process():
  # print("Creating Dataloader")
  train_data_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers, collate_fn=train_dataset.collate_fn,shuffle=False)
  validate_data_loader = DataLoader(validate_dataset, batch_size=batch_size, num_workers=num_workers, collate_fn=test_dataset.collate_fn,shuffle=False)
  test_data_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=num_workers, collate_fn=test_dataset.collate_fn,shuffle=False)




  embedding_dim = hidden
  code2idx_file = '/content/drive/MyDrive/iirlab/topic/LSAN/LSANori/code2idx_new.pickle'



  with open(code2idx_file, 'rb') as f:
      code2idx = pickle.load(f)
      #  list out keys and values separately
      diagnosis_code_list = list(code2idx.keys())
      dignosis_index_list = list(code2idx.values())
      # print(len(dignosis_index_list))

  # model = LSAN(len(dignosis_index_list), embedding_dim, transformer_hidden = hidden, attn_heads = attn_heads,
  #                     transformer_dropout = dropout, transformer_layers = layers)
  print(hidden)
  model = LSAN(51, embedding_dim, transformer_hidden = hidden, attn_heads = attn_heads,
                      transformer_dropout = dropout, transformer_layers = layers)

  # defining trainer
  LSAN_train = LSAN_trainer(model, train_dataloader=train_data_loader, validate_dataloader=validate_data_loader, test_dataloader=test_data_loader, with_cuda=with_cuda, lr=lr, output_dir=output_path)

  # training process
  for epoch in range(epochs):
      model.train()
      LSAN_train.train(epoch)
  
      # Validation
      if validate_data_loader is not None:
          model.eval()
          LSAN_train.validate(epoch)
        
  torch.save(model.state_dict(), saving_path)

  # Test after the end of training
  # test_model = LSAN(len(dignosis_index_list), embedding_dim, transformer_hidden = hidden, attn_heads = attn_heads, transformer_dropout = dropout, transformer_layers = layers) 
  test_model = LSAN(51, embedding_dim, transformer_hidden = hidden, attn_heads = attn_heads, transformer_dropout = dropout, transformer_layers = layers) 


  test_model.load_state_dict(torch.load(saving_path))
  test_model.eval()

  LSAN_test = LSAN_trainer(test_model, train_dataloader=train_data_loader, validate_dataloader=validate_data_loader, test_dataloader=test_data_loader, with_cuda=with_cuda, lr=lr, output_dir=output_path)

  LSAN_test.test(epoch)

In [ ]:
hidden = 2048 
layers = 16
attn_heads = 16
epochs = 100
process()

2048
Done for epoch 0
0.7216295914239242
Validation epoch:0 auc:0.7400000278028004 precision:0.8581765557163531 recall:0.4797734627831715
Validation loss : 0.6246089454282794
Done for epoch 1
0.5985000891094449
Validation epoch:1 auc:0.8105989835296212 precision:0.8210689388071263 recall:0.8576051779935275
Validation loss : 0.4938965931273343
Done for epoch 2
0.5637074189276254
Validation epoch:2 auc:0.8120558502652386 precision:0.8141993957703928 recall:0.872168284789644
Validation loss : 0.48892592547232644
Done for epoch 3
0.5561026349788954
Validation epoch:3 auc:0.8182030494111368 precision:0.8172782874617737 recall:0.8648867313915858
Validation loss : 0.48056528129075704
Done for epoch 4
0.5438560665655536
Validation epoch:4 auc:0.8201687073922086 precision:0.8271213322759714 recall:0.8438511326860841
Validation loss : 0.48445716918560494
Done for epoch 5
0.53466807969478
Validation epoch:5 auc:0.8256375182108342 precision:0.8531781557743957 recall:0.7710355987055016
Validation l

In [ ]:
hidden = 2048 
layers = 16
attn_heads = 16
epochs = 200
process()

In [ ]:
hidden = 4096
layers = 16
attn_heads = 16 
epochs = 100
process()

In [ ]:
hidden = 4096 
layers = 16
attn_heads = 16 
epochs = 200
process()

# 一次跑很多種

In [ ]:
from itertools import product
parameters = dict(
    lr_value = [0.0005, 0.0001],
    batch_size_value = [32, 64],
    hidden_value = [128, 256, 512],
    #awd_value = [0.01, 0.005, 0.001],
    layers_value = [2, 4, 8],
)

param_values = [v for v in parameters.values()]

for lr_value, batch_size_value, hidden_value, layers_value in product(*param_values):
  lr = lr_value
  batch_size = batch_size_value
  hidden = hidden_value
  #adam_weight_decay = awd_value
  layers = layers_value
  comment = f' lr = {lr} batch_size = {batch_size} hidden = {hidden} layers = {layers}'
  print(comment)
  process()

 lr = 0.0005 batch_size = 32 hidden = 128 layers = 2
Testing epoch:19 auc:0.7757754527640222 precision:0.827996340347667 recall:0.7375713121434393 F1:0.7801724137931034
 lr = 0.0005 batch_size = 32 hidden = 128 layers = 4
Testing epoch:19 auc:0.7830828358548563 precision:0.8160919540229885 recall:0.752241238793806 F1:0.7828668363019509
 lr = 0.0005 batch_size = 32 hidden = 128 layers = 8
Testing epoch:19 auc:0.772555460403747 precision:0.8151875571820677 recall:0.726161369193154 F1:0.7681034482758621
 lr = 0.0005 batch_size = 32 hidden = 256 layers = 2
Testing epoch:19 auc:0.7773268409461676 precision:0.8090366581415175 recall:0.7734311328443357 F1:0.7908333333333334
 lr = 0.0005 batch_size = 32 hidden = 256 layers = 4
Testing epoch:19 auc:0.7764952968805376 precision:0.8031173092698933 recall:0.797881010594947 F1:0.8004905968928863
 lr = 0.0005 batch_size = 32 hidden = 256 layers = 8
Testing epoch:19 auc:0.7771324002940053 precision:0.82277318640955 recall:0.7302363488182559 F1:0.7737

In [ ]:
from itertools import product
parameters = dict(
    lr_value = [0.0005, 0.0001],
    batch_size_value = [32, 64],
    hidden_value = [128, 256, 512],
    #awd_value = [0.01, 0.005, 0.001],
    layers_value = [2, 4, 8],
)

param_values = [v for v in parameters.values()]

for lr_value, batch_size_value, hidden_value, layers_value in product(*param_values):
  lr = lr_value
  batch_size = batch_size_value
  hidden = hidden_value
  #adam_weight_decay = awd_value
  layers = layers_value
  comment = f' lr = {lr} batch_size = {batch_size} hidden = {hidden} layers = {layers}'
  print(comment)
  process()

 lr = 0.0005 batch_size = 32 hidden = 128 layers = 2
Testing epoch:19 auc:0.7807040406422333 precision:0.8879456706281834 recall:0.4262428687856561 F1:0.5759911894273128
 lr = 0.0005 batch_size = 32 hidden = 128 layers = 4
Testing epoch:19 auc:0.7782549158320198 precision:0.8927973199329984 recall:0.4343928280358598 F1:0.5844298245614036
 lr = 0.0005 batch_size = 32 hidden = 128 layers = 8
Testing epoch:19 auc:0.7870240513433644 precision:0.9034267912772586 recall:0.47269763651181745 F1:0.6206527554842162
 lr = 0.0005 batch_size = 32 hidden = 256 layers = 2
Testing epoch:19 auc:0.7820913264300007 precision:0.8978930307941653 recall:0.4515077424612877 F1:0.6008676789587853
 lr = 0.0005 batch_size = 32 hidden = 256 layers = 4
Testing epoch:19 auc:0.7871178241401517 precision:0.8956521739130435 recall:0.5036674816625917 F1:0.6447574334898278
 lr = 0.0005 batch_size = 32 hidden = 256 layers = 8
Testing epoch:19 auc:0.7657059092031105 precision:0.8757861635220126 recall:0.4539527302363488 F

KeyboardInterrupt: ignored